In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
data = pd.read_csv("../input/insurance/insurance.csv")
data.head()

In [ ]:
data.describe()

In [ ]:
data.info()

In [ ]:
import seaborn as sns
sns.set()

In [ ]:
sns.distplot(data["children"])

In [ ]:
sns.scatterplot(x="age",y="charges",data=data)

In [ ]:
sns.scatterplot(x="children",y="charges",data=data)

In [ ]:
sns.scatterplot(x="bmi",y="charges",data=data,alpha=0.5)

In [ ]:
log_charges = np.log(data["charges"])
data["log_charges"] = log_charges
data.head()

In [ ]:
sns.scatterplot(x="age",y="log_charges",data=data,alpha=0.5)

In [ ]:
data_d=pd.get_dummies(data,drop_first=True)
data_d.head()

In [ ]:
data_d.columns.values

In [ ]:
cols=['log_charges','age', 'bmi', 'children', 'charges', 'sex_male',
       'smoker_yes', 'region_northwest', 'region_southeast',
       'region_southwest']

In [ ]:
data_r=data_d[cols]
data_r.head()

In [ ]:
data_r=data_r.drop(["charges"],axis=1)
data_r.head()

In [ ]:
target = data_r["log_charges"]
inputs = data_r.drop(["log_charges"],axis=1)

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()
scaler.fit(inputs)

In [ ]:
inputs_scaled = scaler.transform(inputs)

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(inputs_scaled,target,test_size=0.2,random_state=42)

In [ ]:
reg= LinearRegression()
reg.fit(x_train,y_train)

In [ ]:
y_hat = reg.predict(x_train)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.scatter(y_train, y_hat,alpha=0.3)
plt.xlabel('Targets (y_train)')
plt.ylabel('Predictions (y_hat)')
plt.show()

In [ ]:
sns.distplot(y_train-y_hat)

In [ ]:
reg_summary = pd.DataFrame(inputs.columns.values,columns=["Features"])

In [ ]:
reg_summary["weights"]=reg.coef_
reg_summary

In [ ]:
y_hat_test = reg.predict(x_test)

In [ ]:
plt.scatter(y_test, y_hat_test, alpha=0.5)
plt.xlabel('Targets (y_test)',size=18)
plt.ylabel('Predictions (y_hat_test)',size=18)
plt.show()

In [ ]:
df_pf = pd.DataFrame(np.exp(y_hat_test), columns=['Prediction'])
df_pf.head()

In [ ]:
y_test=y_test.reset_index(drop=True)

In [ ]:
df_pf['Target'] = np.exp(y_test)
df_pf

In [ ]:
df_pf['Residual'] = df_pf['Target'] - df_pf['Prediction']
df_pf['Difference%'] = np.absolute(df_pf['Residual']/df_pf['Target']*100)
df_pf

In [ ]:
df_pf.describe()

In [ ]:
pd.options.display.max_rows = 999
pd.set_option('display.float_format', lambda x: '%.2f' % x)
df_pf.sort_values(by=['Difference%'])